In [ ]:
!pip install datasets
!pip install accelerate
!pip install peft
!pip install japanize_matplotlib

In [ ]:
from datasets import list_datasets

In [ ]:
all_datasets = list_datasets()
all_datasets

In [ ]:
from datasets import load_dataset
emotions = load_dataset("dair-ai/emotion")
emotions

In [ ]:
type(emotions)

In [ ]:
emotions["train"]

In [ ]:
emotions["train"][0]

In [ ]:
emotions["train"].features

In [ ]:
# import pandas as pd
# emotions.set_format(type="pandas")
# df = emotions["train"][:]
# df

In [ ]:
# def label_int2str(row):
#   return emotions["train"].features["label"].int2str(row)

# df["label_name"] = df["label"].apply(label_int2str)

In [ ]:
# df

In [ ]:
from transformers import AutoTokenizer

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize(batch):
  token = tokenizer(batch["text"], padding=True)
  return token

In [ ]:
emotions_tokenized = emotions.map(tokenize, batched=True, batch_size=None)
emotions_tokenized

In [ ]:
emotions_tokenized["train"]

In [ ]:
emotions_tokenized["train"].features

In [ ]:
from transformers import AutoModelForSequenceClassification
import numpy as np
import os

In [ ]:
model_name

In [ ]:
num_labels = len(emotions["train"].features["label"].names)
num_labels

In [ ]:
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
base_model

In [ ]:
import peft

In [ ]:
peft_config = peft.LoraConfig(
    task_type=peft.TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    inference_mode=False,
    target_modules="all-linear",
)

In [ ]:
model = peft.get_peft_model(base_model, peft_config)
model

In [ ]:
model.print_trainable_parameters()

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir = "./finetuned_result",
    evaluation_strategy = "epoch",
    learning_rate = 1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay = 0.01,
    push_to_hub = False
    )

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
def metrics(pred):
  labels = pred.label_ids
  predictions = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, predictions)
  f1 = f1_score(labels, predictions, average = "weighted")
  return {"accuracy":acc, "f1":f1}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=emotions_tokenized["train"],
    eval_dataset=emotions_tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=metrics
)

In [ ]:
trainer.train()

In [ ]:
pred_output = trainer.predict(emotions_tokenized["test"])
pred_output

In [ ]:
pred_output.predictions

In [ ]:
y_pred = np.argmax(pred_output.predictions, axis=1)
y_test = np.array(emotions_tokenized["test"]["label"])
label_name = emotions_tokenized["test"].features["label"].names

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import japanize_matplotlib

In [ ]:
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8,8))
dp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = label_name)
dp.plot(ax=ax, colorbar=False)
plt.title('混同行列')
plt.show()